In [ ]:
import pandas as pd
from pymongo import MongoClient
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale, normalize

db = MongoClient('localhost', 27017)
cursor = db.sustaindb['macav2'].find()
df = pd.DataFrame(list(cursor))
df

In [2]:
features = [
    'E_HU',
    'M_HU',
    'E_HH',
    'M_HH',
    'E_POV',
    'M_POV',
    'E_UNEMP',
    'M_UNEMP',
    'E_PCI',
    'M_PCI',
    'E_NOHSDP',
    'M_NOHSDP',
    'E_AGE65',
    'M_AGE65',
    'E_AGE17',
    'M_AGE17',
    'E_DISABL',
    'M_DISABL',
    'E_SNGPNT',
    'M_SNGPNT',
    'E_MINRTY',
    'M_MINRTY',
    'E_LIMENG',
    'M_LIMENG',
    'E_MUNIT',
    'M_MUNIT',
    'E_MOBILE',
    'M_MOBILE',
    'E_CROWD',
    'M_CROWD',
    'E_NOVEH',
    'M_NOVEH',
    'E_GROUPQ',
    'M_GROUPQ',
]

print(f'#features:{len(features)}')

x = df.loc[:, features]

y = df.loc[:, 'RPL_THEMES']

x.loc[:,:] = normalize(x)

#features:34


In [3]:
# df.loc[:,:] = StandardScaler().fit_transform(df)

In [12]:
total_variance = x.var().sum()
total_variance

0.20879673247631805

In [14]:
#top 10 features with variance
feature_var = (x.var()/x.var().sum()).sort_values(ascending=False)
feature_var.to_json()

'{"E_PCI":0.4133199259,"E_MINRTY":0.1665853802,"E_HU":0.1488724892,"E_HH":0.1179051985,"E_AGE17":0.0465882754,"E_POV":0.0238098832,"E_AGE65":0.0231011522,"E_DISABL":0.0187603067,"M_PCI":0.0095294145,"E_NOHSDP":0.0087111876,"E_MOBILE":0.0067054438,"E_GROUPQ":0.0054907404,"E_MUNIT":0.0041067978,"E_LIMENG":0.0023245787,"E_NOVEH":0.0012374743,"E_SNGPNT":0.0011358619,"E_UNEMP":0.0008893471,"M_POV":0.0002606791,"E_CROWD":0.0001676545,"M_DISABL":0.0000890261,"M_GROUPQ":0.0000781494,"M_NOHSDP":0.000068141,"M_MOBILE":0.0000527518,"M_HH":0.0000444286,"M_LIMENG":0.0000242772,"M_UNEMP":0.0000235051,"M_MINRTY":0.0000234943,"M_HU":0.0000185553,"M_SNGPNT":0.0000183411,"M_MUNIT":0.0000152114,"M_NOVEH":0.0000149283,"M_AGE17":0.0000115528,"M_AGE65":0.0000083017,"M_CROWD":0.0000075448}'

In [15]:
from sklearn.decomposition import PCA

pca = PCA(n_components=len(features), random_state=2021)

# principalComponents = pca.fit_transform(x)
x_pca_34 = pca.fit_transform(x)

print(f'Variance explained by all 34 PCs = {pca.explained_variance_ratio_}')
print(len(pca.explained_variance_ratio_))

Variance explained by all 34 PCs = [7.99615266e-01 1.22285073e-01 3.31151156e-02 1.25230990e-02
 9.54510923e-03 5.52444846e-03 4.02934552e-03 3.25087286e-03
 2.76697010e-03 2.07772088e-03 1.67062939e-03 1.28562590e-03
 8.52745422e-04 5.87387611e-04 3.72210494e-04 1.41902980e-04
 1.11985422e-04 7.30400975e-05 5.01869269e-05 3.79854036e-05
 2.27784522e-05 1.14405350e-05 8.32077575e-06 8.00051176e-06
 6.46940206e-06 6.04700868e-06 4.30857798e-06 3.60404997e-06
 2.90688140e-06 2.67491162e-06 2.31252079e-06 1.87714045e-06
 1.37026740e-06 1.16836284e-06]
34


In [18]:
pcs = pd.DataFrame(pca.components_,columns=features, 
                   index = [f'pc_{x}' for x in range(1,len(features)+1)])
pcs.to_json()

'{"E_HU":{"pc_1":0.3866468414,"pc_2":-0.4380260613,"pc_3":-0.2487365053,"pc_4":-0.1125736847,"pc_5":0.545973281,"pc_6":-0.0377454138,"pc_7":0.1749403059,"pc_8":0.3705488533,"pc_9":-0.2101615532,"pc_10":0.0890929578,"pc_11":-0.236461929,"pc_12":-0.0639652567,"pc_13":-0.0589420311,"pc_14":-0.0102669924,"pc_15":0.0197627628,"pc_16":-0.0096193026,"pc_17":0.0004119201,"pc_18":-0.0148199086,"pc_19":-0.0069784163,"pc_20":-0.0045826693,"pc_21":0.0011738333,"pc_22":-0.0059654204,"pc_23":-0.0016181318,"pc_24":-0.0001755713,"pc_25":-0.0090617231,"pc_26":-0.0077239858,"pc_27":-0.0030234329,"pc_28":0.000023153,"pc_29":-0.0020196744,"pc_30":0.0012712472,"pc_31":0.0031405291,"pc_32":0.0009180064,"pc_33":0.0014394638,"pc_34":-0.0004453918},"M_HU":{"pc_1":-0.0008537838,"pc_2":-0.0003437367,"pc_3":-0.0062567724,"pc_4":0.0038268238,"pc_5":0.0051462764,"pc_6":0.0254017101,"pc_7":-0.0216051305,"pc_8":0.0086469909,"pc_9":-0.0034110031,"pc_10":-0.0013049422,"pc_11":-0.0058466795,"pc_12":0.0002492606,"pc_13":

In [7]:
variance_df = pd.DataFrame(pca.explained_variance_ratio_*100/pca.explained_variance_ratio_.sum(),index=x.columns.tolist(),columns=['percentage'],)
variance_df['cum_sum'] = variance_df.cumsum()

In [8]:
variance_df

,percentage,cum_sum
E_HU,79.961527,79.961527
M_HU,12.228507,92.190034
E_HH,3.311512,95.501545
M_HH,1.252310,96.753855
E_POV,0.954511,97.708366
M_POV,0.552445,98.260811
E_UNEMP,0.402935,98.663746
M_UNEMP,0.325087,98.988833
E_PCI,0.276697,99.265530
M_PCI,0.207772,99.473302
